# Evaporation - Groundwater Recharge

In [62]:
# read all necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, Button, VBox, HBox, widgets, interact
from IPython.display import display, clear_output


### Variables, Acronyms
ETP...Potential Evapotranspiration [mm/d] \
ETR...Real(actual) Evapotranspiration [mm/d] \
Re...extraterrestrial radiation [MJ/(m^2 d)] \
S_B_A...The volumetric soil moisture [%] \
awc...available water capacity [mm] \
K_s...water stress coefficient [-] \
PP...Precipitation [mm] \
Re_mm_day...Extraterrestrial radiation [mm/d]\
xrain... \
p...precipitation

In [63]:
# load data from ETP_data.csv
data = pd.read_csv('ETP_data.csv', index_col='datum', parse_dates=True)
ETP = data.ETP
Re = data.Re


In [66]:
class SoilMoistureSimulation:
    def __init__(self, data):
        self.data = data
        self.awc = 150
        self.p = 0.7
        self.xrain = 1
        self.ref_data = None
        #self.create_interactive_plot()

    def update_moisture_content(self, soilwater_content, PP, ETR, awc):
        if soilwater_content + PP - ETR > awc:
            updated_value = awc
            excess_water = soilwater_content + PP - ETR - awc
        else:
            updated_value = soilwater_content + PP - ETR
            excess_water = 0
        return updated_value, excess_water

    def linear_reduction_function(self, S_B_A, available_water_capacity, fraction_of_awc=0.7):
        theta_0 = available_water_capacity * fraction_of_awc
        if S_B_A >= theta_0:
            K_s = 1
        elif S_B_A <= 0:
            K_s = 0
        else:
            K_s = S_B_A / theta_0
        return K_s

    def run_simulation(self, awc=150, p=0.7, xrain=1):
        ETP = self.data['ETP']  # Assuming 'ETP' is a column in your 'data'
        Precip = self.data['N (mm/d)'] * xrain
        gw_flux = np.zeros(len(self.data))
        soil_moisture = np.zeros(len(self.data))
        soil_moisture[0] = awc
        ETR = np.zeros(len(self.data))
        
        for i in range(1, len(self.data)):
            ETR[i] = ETP[i] * self.linear_reduction_function(soil_moisture[i-1], awc, p)
            soil_moisture[i], excess_water = self.update_moisture_content(soil_moisture[i-1], Precip[i], ETR[i], awc)
            gw_flux[i] = excess_water

        return Precip, ETP, ETR, soil_moisture, gw_flux

    def plot_simulation(self, awc, p, xrain):
        # Running the current simulation
        Precip, ETP, ETR, soil_moisture, gw_flux = self.run_simulation(awc, p, xrain)
        
        # If there's reference data, unpack it
        if self.ref_data:
            ref_precip, ref_etp, ref_etr, ref_soil_moisture, ref_gw_flux = self.ref_data
        
        # Plotting setup
        fig = plt.figure(figsize=(12, 7))
        width_ratio = 0.4
        spacing_vertical = 0.05
        spacing_horizontal = 0.09
        
        left_plot_width = (1 - width_ratio - spacing_horizontal)
        right_plot_width = width_ratio
        
        height_left = (1 - 4 * spacing_vertical) / 3
        positions_left = [2/3 + spacing_vertical, 1/3 + spacing_vertical, 0 + spacing_vertical]
        
        height_right = (1 - 2 * spacing_vertical) / 2
        positions_right = [1/2 + spacing_vertical/2, 0 + spacing_vertical - 0.05]
        
        # Subplot axes
        ax1 = fig.add_axes([0, positions_left[0], left_plot_width, height_left])
        ax2 = fig.add_axes([0, positions_left[1], left_plot_width, height_left])
        ax3 = fig.add_axes([0, positions_left[2], left_plot_width, height_left])
        ax_right1 = fig.add_axes([1 - right_plot_width, positions_right[0], right_plot_width, height_right])
        ax_right2 = fig.add_axes([1 - right_plot_width, positions_right[1], right_plot_width, height_right])
        
        # Plot current data
        ax1.bar(self.data.index, Precip, color='skyblue', label='Precipitation', width=1)
        ax2.plot(self.data.index, ETP, color='goldenrod', label='ETP')
        ax2.plot(self.data.index, ETR, color='tomato', label='ETR')
        ax3.plot(self.data.index, soil_moisture, color='steelblue', label='Soil Moisture')
        ax3_twin = ax3.twinx()
        ax3_twin.bar(self.data.index, gw_flux, color='navy', label='Groundwater Flux', width=1)

        # Plot cumulative current data
        ax_right1.plot(self.data.index, np.cumsum(Precip), color='skyblue', label='Cumulative Precipitation')
        ax_right1.plot(self.data.index, np.cumsum(gw_flux), color='navy', label='Cumulative GW Flux')
        ax_right2.plot(self.data.index, np.cumsum(ETP), color='goldenrod', label='Cumulative ETP')
        ax_right2.plot(self.data.index, np.cumsum(ETR), color='tomato', label='Cumulative ETR')
        
        # Plot reference data if available
        if self.ref_data:
            #ax1.plot(self.data.index, ref_precip, 'skyblue', linestyle='--', label='Ref Precipitation')
            #ax2.plot(self.data.index, ref_etp, 'goldenrod', linestyle='--', label='Ref ETP')
            ax2.plot(self.data.index, ref_etr, 'tomato', linestyle='--', label='Ref ETR')
            ax3.plot(self.data.index, ref_soil_moisture, 'steelblue', linestyle='--', label='Ref Soil Moisture')
            #ax3.twinx().plot(self.data.index, ref_gw_flux, 'navy', linestyle='--', label='Ref GW Flux')

            # Plot cumulative reference data
            ax_right1.plot(self.data.index, np.cumsum(ref_precip), 'skyblue', linestyle='--', label='Ref Cumulative Precipitation')
            ax_right1.plot(self.data.index, np.cumsum(ref_gw_flux), 'navy', linestyle='--', label='Ref Cumulative GW Flux')
            #ax_right2.plot(self.data.index, np.cumsum(ref_etp), 'goldenrod', linestyle='--', label='Ref Cumulative ETP')
            ax_right2.plot(self.data.index, np.cumsum(ref_etr), 'tomato', linestyle='--', label='Ref Cumulative ETR')

        x = True

        # convert axis to log if x is true
        if x:
            ax1.set_yscale('log')
            ax2.set_yscale('log')
            ax3.set_yscale('log')
            ax_right1.set_yscale('log')
            ax_right2.set_yscale('log')

        # Adding labels, legends, and finalizing plot layout
        ax1.set_ylabel('Precipitation (mm/d)')
        ax2.set_ylabel('ETP and ETR (mm/d)')
        ax3.set_ylabel('Soil Moisture (mm)')
        ax3_twin.set_ylabel('Groundwater Flux (mm/d)')
        ax_right1.set_ylabel('Cumulative Precipitation and GW Flux (mm)')
        ax_right2.set_ylabel('Cumulative ETP and ETR (mm)')

        for ax in [ax1, ax2, ax3, ax_right1, ax_right2, ax3_twin]:
            ax.legend(loc='best')
        
        plt.tight_layout()

    def on_save_click(self, button):
        self.ref_data = self.run_simulation(self.awc, self.p, self.xrain)
        print("Reference state saved.")

    def create_interactive_plot(self):
        awc_slider = widgets.FloatSlider(min=0, max=500, step=5, value=self.awc, description='AWC:')
        p_slider = widgets.FloatSlider(min=0.5, max=1.0, step=0.1, value=self.p, description='p:')
        xrain_slider = widgets.FloatSlider(min=0.1, max=5, step=0.1, value=self.xrain, description='xRain:')

        save_btn = widgets.Button(description="Save Current State")
        save_btn.on_click(self.on_save_click)

        def update_plot(awc, p, xrain):
            self.plot_simulation(awc, p, xrain)
            if self.ref_data:
                # Optionally handle plotting of the reference data here
                pass

        ui = widgets.VBox([awc_slider, p_slider, xrain_slider, save_btn])
        out = widgets.interactive_output(update_plot, {'awc': awc_slider, 'p': p_slider, 'xrain': xrain_slider})

        display(ui, out)


plotter = SoilMoistureSimulation(data)


In [71]:
plotter.create_interactive_plot()

Output()

In [68]:
class SoilMoistureSimulation:
    def __init__(self, variable):
        self.var = variable
        self.ref_data = None

    def add_value(self, x):

        current_data = self.var + x
        self.ref_data = current_data
        return current_data


In [69]:
class_test = SoilMoistureSimulation(5)

In [70]:
class_test.add_value(3)

8